# HW 2 Part 1

By: Chengyi (Jeff) Chen

In [1]:
%load_ext nb_black
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.figsize"] = (16, 12)

import pandas as pd
import numpy as np
import cvxpy as cp
import scipy as sp
from collections import namedtuple
from statsmodels.graphics.gofplots import qqplot
from datetime import datetime, timedelta
from tqdm import tqdm

/opt/anaconda3/envs/ml/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<IPython.core.display.Javascript object>

#### SPY and US Dollar Index Monthly Returns

In [2]:
spy_monthly_returns = pd.read_excel(
    "./data/HW2.xlsx", sheet_name=2, usecols=[0, 1], header=[0], index_col=[0]
)
spy_monthly_returns.index = pd.to_datetime(spy_monthly_returns.index)
spy_monthly_returns.head()

,S&P - 500 Index
2001-07-31,-0.984416
2001-08-31,-6.260163
2001-09-30,-8.075231
2001-10-31,1.906875
2001-11-30,7.670629


<IPython.core.display.Javascript object>

In [3]:
us_dollar_monthly_returns = pd.read_excel(
    "./data/HW2.xlsx", sheet_name=2, usecols=[0, 2], header=[0], index_col=[0]
)
us_dollar_monthly_returns.index = pd.to_datetime(us_dollar_monthly_returns.index)
us_dollar_monthly_returns.head()

,US Dollar Index
2001-07-31,-1.916799
2001-08-31,-3.208739
2001-09-30,-0.008817
2001-10-31,1.278547
2001-11-30,1.105694


<IPython.core.display.Javascript object>

#### S&P 500 Equities Monthly Returns

In [4]:
equity_monthly_returns = (
    pd.read_excel("./data/HW2.xlsx", sheet_name=1, header=[1]).iloc[:, 4:].T
)
equity_monthly_returns.columns = pd.MultiIndex.from_frame(
    pd.read_excel("./data/HW2.xlsx", sheet_name=1, header=[1], usecols=[0, 1, 2, 3])
)
equity_monthly_returns.index = pd.to_datetime(equity_monthly_returns.index)
equity_monthly_returns.head()

NAME,3m Co,Abbott Labs,Abbvie Inc,Abiomed Inc,Accenture Plc Ireland,Activision Blizzard Inc,Adobe Sys Inc,Advance Auto Parts,Advanced Micro Devic,Aes Corp,...,Willis Towers Watson Pu,Wynn Resorts Ltd,Xcel Energy Inc,Xerox Corp,Xilinx Inc,Xylem Inc,Yum Brands Inc,Zimmer Hldgs Inc,Zions Bancorp,Zoetis Inc
TICKER,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AAP,AMD,AES,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
SECTOR,INDU,HLTH,HLTH,HLTH,INFT,TCOM,INFT,DSCR,INFT,UTIL,...,FINA,DSCR,UTIL,INFT,INFT,INDU,DSCR,HLTH,FINA,HLTH
Market Cap ($ Mil),110949,127036,138674,14640,89887,35535,110435,11478,18449,9577,...,19733,10755,25326,4708,21552,11991,28707,21156,7830,41098
2001-07-31,-1.945100,12.112976,NaN,-19.423240,NaN,-13.577718,-20.234087,NaN,-36.816609,-11.033682,...,-2.816904,NaN,-4.077114,-16.614480,-3.006790,NaN,4.214608,NaN,-0.932129,NaN
2001-08-31,-6.446533,-7.258951,NaN,-1.315789,-0.401070,9.226306,-10.349165,NaN,-25.794085,-13.524804,...,9.043619,NaN,1.707327,15.288163,-2.400000,NaN,-6.842003,-4.895105,-1.698511,NaN
2001-09-30,-5.475902,4.326061,NaN,-6.826667,-14.429530,-26.532794,-28.616782,NaN,-39.852399,-61.292271,...,24.348706,NaN,4.106134,-15.760939,-39.728484,NaN,-7.977550,2.022059,-6.287271,NaN
2001-10-31,6.077282,2.599966,NaN,24.441900,37.803922,32.808711,10.091781,NaN,20.736196,8.034321,...,-0.427533,NaN,0.461629,-9.677419,29.281768,NaN,28.990256,11.387387,-10.696854,NaN
2001-11-30,10.339559,3.812730,NaN,-12.741490,28.628344,3.113693,21.515301,NaN,37.804878,19.277978,...,1.674522,NaN,-3.429815,20.000215,18.704799,NaN,-6.206153,4.367519,1.164570,NaN


<IPython.core.display.Javascript object>

---
## A - Calculate historical factor monthly returns for the following factors based on Arbitrage Pricing Theory

1. Equity market factor. Using SP500 index return as equity market return proxy. (Data see SPX_DXY_MonthlyReturns)
2. US Dollar factor. Using  DXY Index returns as proxy. (Data see SPX_DXY_MonthlyReturns)
3. Sector factors. (Note: you should leave one sector out to make sure proper running of Fama Macbeth regression. Let's leave out the sector of UTIL.)
4. Size factor. At each month-end reconstitution date, we use the same company market cap data (column D on SP500Stocks) for simplicity. (Note: size exposure for a company should be sector neutral, i.e., the z-score of the log(mkt_cap) should be w.r.t. peer companies within the same sector.)

OLS Closed Form Solution:
\begin{align}
    \hat\beta &= (X^TX)^{-1}X^Ty
\end{align}

In [5]:
intersect = lambda a, b: sorted(set(a) & set(b))


def β_hat(X, y, dropna=True):
    if dropna:
        X, y = X.dropna(), y.dropna()
        common_index = intersect(X.index, y.index)
        X, y = X.loc[common_index], y.loc[common_index]
    else:
        X, y = X.fillna(0.0), y.fillna(0.0)
    return (
        pd.DataFrame(np.linalg.inv(X.T @ X), columns=X.columns, index=X.columns)
        @ X.T
        @ y.values
    )

<IPython.core.display.Javascript object>

## [Fama-Macbeth Two-step Regression](http://didattica.unibocconi.it/mypage/dwload.php?nomefile=fama-macbeth20141115121157.pdf)

### Step 1 - Regress the Time Series of Returns of the Stock against Time Series of Factors to get Factor Exposures $\beta_{\text{factor}}$

For $m$ factors:

\begin{align}
    R_{\text{Stock A}} &= \alpha + \beta_{F_1}F_1 + \beta_{F_2}F_2 + \cdots + \beta_{F_m}F_m + \epsilon
\end{align}

#### Factor Exposures of each stock to S&P 500 and US Dollar

In [6]:
factor_exposures = equity_monthly_returns.apply(
    lambda equity: β_hat(
        X=pd.concat(
            [
                spy_monthly_returns,
                us_dollar_monthly_returns,
                pd.Series(1, name="α", index=spy_monthly_returns.index),
            ],
            axis=1,
        ),
        y=equity,
        dropna=True,
    ),
    axis=0,
).T
factor_exposures.reset_index(level=[1, 2, 3], drop=True, inplace=True)
factor_exposures.rename(
    {
        "S&P - 500 Index": "β (equity market factor exposure)",
        "US Dollar Index": "β (us dollar factor exposure)",
    },
    axis=1,
    inplace=True,
)
factor_exposures.head()

,β (equity market factor exposure),β (us dollar factor exposure),α
NAME,,,
3m Co,0.897279,0.137059,0.411085
Abbott Labs,0.501031,-0.096830,0.635259
Abbvie Inc,1.450096,-0.556901,0.662926
Abiomed Inc,1.263956,0.059849,1.770226
Accenture Plc Ireland,1.152855,0.193686,0.772685


<IPython.core.display.Javascript object>

#### Sector factor one-hot encoding, drop UTIL sector column

In [7]:
sector_factor = pd.concat(
    [
        pd.Series(equity_monthly_returns.columns.get_level_values("NAME")),
        pd.Series(equity_monthly_returns.columns.get_level_values("SECTOR")),
    ],
    axis=1,
)
sector_factor.set_index("NAME", inplace=True)
sector_factor = pd.get_dummies(sector_factor).drop(["SECTOR_UTIL"], axis=1)
sector_factor.head()

,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM
NAME,,,,,,,,,,
3m Co,0,0,0,0,1,0,0,0,0,0
Abbott Labs,0,0,0,1,0,0,0,0,0,0
Abbvie Inc,0,0,0,1,0,0,0,0,0,0
Abiomed Inc,0,0,0,1,0,0,0,0,0,0
Accenture Plc Ireland,0,0,0,0,0,1,0,0,0,0


<IPython.core.display.Javascript object>

#### Size factor - Standardize log(Market Cap) by sector

In [8]:
size_factor = (
    pd.concat(
        [
            pd.Series(equity_monthly_returns.columns.get_level_values("NAME")),
            pd.Series(equity_monthly_returns.columns.get_level_values("SECTOR")),
            pd.Series(
                equity_monthly_returns.columns.get_level_values("Market Cap ($ Mil)")
            ),
        ],
        axis=1,
    )
    .groupby("SECTOR")["Market Cap ($ Mil)"]
    .transform(lambda x: (np.log(x) - np.log(x).mean()) / np.log(x).std())
    .to_frame()
)
size_factor.index = equity_monthly_returns.columns.get_level_values("NAME")
size_factor.rename(
    {"Market Cap ($ Mil)": "β (size factor exposure)"}, axis=1, inplace=True
)
size_factor.head()

,β (size factor exposure)
NAME,
3m Co,1.977538
Abbott Labs,1.350510
Abbvie Inc,1.436935
Abiomed Inc,-0.779869
Accenture Plc Ireland,1.038986


<IPython.core.display.Javascript object>

### Step 2 - Regress Returns of all the Stock in the Portfolio @ time $t, t \in T$ against Factor Exposures $\beta_{\text{factor}}$ of each Stock

For $t \in T$ timesteps:

\begin{align}
    R_{\text{Stock }i, t} &= \gamma_{t, F_0} + \gamma_{t, F_1}\hat{\beta_{F_1}} + \gamma_{t, F_2}\hat{\beta_{F_2}} + \cdots + \gamma_{t, F_m}\hat{\beta_{F_m}} + \epsilon
\end{align}

In [9]:
γ_pure_factor_monthly_returns = equity_monthly_returns.apply(
    lambda returns: β_hat(
        X=pd.concat(
            [
                factor_exposures.drop(["α"], axis=1),
                sector_factor,
                size_factor,
                pd.Series(1, index=size_factor.index, name="γ_residual"),
            ],
            axis=1,
        ),
        y=returns,
        dropna=False,
    ),
    axis=1,
)
γ_pure_factor_monthly_returns.head()

,β (equity market factor exposure),β (us dollar factor exposure),SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM,β (size factor exposure),γ_residual
2001-07-31,-5.081906,0.546521,9.689164,4.497985,6.003830,3.868261,7.997561,5.971431,7.795002,3.298414,5.834931,4.522445,-0.666501,-0.172946
2001-08-31,-5.351213,-2.530621,-0.865683,-1.974351,-0.172676,0.152704,0.283117,-0.786473,2.069103,2.237107,0.399686,-0.210663,-0.887846,3.563947
2001-09-30,-16.448408,-4.242000,6.115425,5.129572,15.719172,7.981975,9.199334,8.180248,9.807025,11.983056,5.616929,8.494224,-0.404314,0.634008
2001-10-31,7.730491,4.661245,-0.237472,5.161065,-8.600545,2.401637,-3.667805,5.580222,-2.020707,-6.257782,-0.408720,0.168053,-0.612423,-2.110206
2001-11-30,9.140003,4.696506,3.084856,-7.999442,-0.934668,0.446165,1.300240,2.209795,3.643307,-0.218926,2.664973,-1.297612,1.092705,-3.667960


<IPython.core.display.Javascript object>

---
## B - Check for each factor their historical returns are significant or not (based on T-stat)

In [10]:
T = γ_pure_factor_monthly_returns.shape[0]

γ_pure_factor_risk_premium = γ_pure_factor_monthly_returns.mean(axis=0)

γ_pure_factor_standard_error = γ_pure_factor_monthly_returns.std(axis=0) / np.sqrt(T)

t_statistic = γ_pure_factor_risk_premium / γ_pure_factor_standard_error

<IPython.core.display.Javascript object>

In [11]:
# One-sample t-test, df = n - 1
p = 1 - sp.stats.t.cdf(t_statistic, df=T - 1)

<IPython.core.display.Javascript object>

In [12]:
α = 0.95  # Confidence level
for pure_factor, p_val in zip(γ_pure_factor_risk_premium.index[:-1], p[:-1]):
    if p_val < 1 - α:
        print(f"{pure_factor} monthly risk premium is statistically significant.")
    else:
        print(f"{pure_factor} monthly risk premium is not statistically significant.")

β (equity market factor exposure) monthly risk premium is not statistically significant.
β (us dollar factor exposure) monthly risk premium is not statistically significant.
SECTOR_DSCR monthly risk premium is not statistically significant.
SECTOR_ENER monthly risk premium is not statistically significant.
SECTOR_FINA monthly risk premium is not statistically significant.
SECTOR_HLTH monthly risk premium is not statistically significant.
SECTOR_INDU monthly risk premium is not statistically significant.
SECTOR_INFT monthly risk premium is not statistically significant.
SECTOR_MATS monthly risk premium is not statistically significant.
SECTOR_REAL monthly risk premium is not statistically significant.
SECTOR_STPL monthly risk premium is not statistically significant.
SECTOR_TCOM monthly risk premium is not statistically significant.
β (size factor exposure) monthly risk premium is statistically significant.


<IPython.core.display.Javascript object>

---
## C - Using the last month in the back-test, i.e., 12/31/2018, show

### 1. All factor portfolios are long-short neutral portfolio, i.e., the total weights sum to 0	

In [13]:
X = pd.concat(
    [
        factor_exposures.drop(["α"], axis=1),
        sector_factor,
        size_factor,
        pd.Series(1, index=size_factor.index, name="γ_residual"),
    ],
    axis=1,
)
y = equity_monthly_returns.loc["2018-12-31"]
pure_factor_portfolio_weights = np.linalg.inv(X.T @ X) @ X.T
pure_factor_portfolio_weights.index = X.columns

<IPython.core.display.Javascript object>

Check that the allocation of portfolio weights for each stock sums to 0 for all pure factor portfolios

In [14]:
# Exclude γ_residual
np.allclose(pure_factor_portfolio_weights.sum(axis=1)[:-1], 0)

True

<IPython.core.display.Javascript object>

### 2. For any factor portfolio, it has unit exposure to its own factor, but zero exposure to all other factors in the model

In [15]:
for pure_factor in X.columns[:-1]:
    stock_exposures_to_pure_factors = X[pure_factor]
    title = "|| " + pure_factor + " ||"
    print("=" * len(title))
    print(title)
    print("=" * len(title))
    for (
        name,
        pure_factor_portfolio_weights_for_one_factor,
    ) in pure_factor_portfolio_weights.iterrows():
        if (
            np.allclose(
                stock_exposures_to_pure_factors.T
                @ pure_factor_portfolio_weights_for_one_factor,
                1,
            )
            and name == pure_factor
        ):
            print(
                f"{pure_factor} has unit exposure to its own factor {name} - factor portfolio weights * stock exposure to factor == 1"
            )
        elif (
            np.allclose(
                stock_exposures_to_pure_factors.T
                @ pure_factor_portfolio_weights_for_one_factor,
                0,
            )
            and name != pure_factor
        ):
            print(
                f"{pure_factor} has 0 exposure to {name} factor - factor portfolio weights * stock exposure to factor == 1"
            )
        else:
            print("Something's wrong")
    print("\n")

|| β (equity market factor exposure) ||
β (equity market factor exposure) has unit exposure to its own factor β (equity market factor exposure) - factor portfolio weights * stock exposure to factor == 1
β (equity market factor exposure) has 0 exposure to β (us dollar factor exposure) factor - factor portfolio weights * stock exposure to factor == 1
β (equity market factor exposure) has 0 exposure to SECTOR_DSCR factor - factor portfolio weights * stock exposure to factor == 1
β (equity market factor exposure) has 0 exposure to SECTOR_ENER factor - factor portfolio weights * stock exposure to factor == 1
β (equity market factor exposure) has 0 exposure to SECTOR_FINA factor - factor portfolio weights * stock exposure to factor == 1
β (equity market factor exposure) has 0 exposure to SECTOR_HLTH factor - factor portfolio weights * stock exposure to factor == 1
β (equity market factor exposure) has 0 exposure to SECTOR_INDU factor - factor portfolio weights * stock exposure to factor == 1

<IPython.core.display.Javascript object>